In [43]:
! pip install langchain

In [ ]:
import os
import openai
import sys
sys.path.append('../..')

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.environ['OPENAI_API_KEY']

In [ ]:
! pip install pypdf 

In [ ]:
from langchain.document_loaders import PyPDFLoader
pdf_material_loader = PyPDFLoader("./docs/Atomic Design.pdf")
pages = pdf_material_loader.load()

In [ ]:
from langchain.document_loaders import WebBaseLoader
course_loader = WebBaseLoader("https://www.muya.et/courses")

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [ ]:
course_doc = course_loader.load()
print(course_doc[0])

page_content='Muyalogy | Learn online course with certificateCoursesአማCategoriesAll CoursesAll CoursesBest sellerLearn English: Upgrade Your Speaking and Listening SkillA course by : Nabil Emil Are you struggling to communicate in English? We understand that it can be difficult to speak naturally in a new language, which is why we have developed an online course to help you master the art of conversation in English. Our course is designed for non-English speakers who want to expand their vocabulary and communicate effectively in everyday situations, whether it be making friends, conducting business, watching English films, or traveling to an English-speaking country.As the world's universal language, having a basic understanding of English is essential for functioning in most countries. However, our course will take your English skills to the next level, enabling you to communicate like a native and express even your most complex thoughts. Our comprehensive curriculum covers all aspect

In [ ]:
faq_loader = WebBaseLoader("https://www.muya.et/content/faq")

In [ ]:
faqs = faq_loader.load()
print(faqs[0])

page_content='Muyalogy | Learn online course with certificateCoursesአማFrequently asked questionWhat is Muyalogy?Muyalogy is an innovative full-service learning platform that produces, markets and manages online courses in a variety of fields. The name Muyalogy is a combination of two words – Muya, which means skill in Amharic, and logy, which means study in Greek. and our platform has currently started registration for the various courses that we are providing. Muyalogy is a place where anyone can learn a new skill.What kind of courses do you offer?Muyalogy provides various skill courses like photography, graphics design, fashion design, music production, entrepreneurship, interior design, guitar, cake baking, nail & lash..and more, Course difficulty for each course is mentioned in the course description.You can find our course catalog on our website muyalogy.comYou can suggest courses along with instructors through our Telegram, email, or phone number.What are the benefits of learning

In [ ]:
len(faqs[0].page_content)

8179

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
r_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=150
)

In [ ]:
splits = r_splitter.split_documents(pages)

In [ ]:
! pip install -U langchain-openai 
from langchain_openai import OpenAIEmbeddings
embedding = OpenAIEmbeddings()

In [ ]:
! pip install chromadb

In [ ]:
from langchain.vectorstores import Chroma

In [ ]:
!rm -rf ./docs/chroma  # remove old database files if any

In [ ]:
persist_directory = 'docs/chroma/'

In [ ]:
vectordb = Chroma.from_documents(
    documents=splits,
    embedding=embedding,
    persist_directory=persist_directory
)

In [ ]:
print(vectordb._collection.count())

262


In [ ]:
question = "Whats is A manageable strategy?"

In [ ]:
docs = vectordb.similarity_search(question,k=5)

In [ ]:
print(vectordb._collection.count())

262


In [ ]:
from langchain_openai import OpenAI
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor

In [ ]:
def pretty_print_docs(docs):
    print(f"\n{'-' * 100}\n".join([f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]))


In [ ]:
# Wrap our vectorstore
llm = OpenAI(temperature=0, model="gpt-3.5-turbo-instruct")
compressor = LLMChainExtractor.from_llm(llm)

In [ ]:
compression_retriever = ContextualCompressionRetriever(
   base_compressor=compressor,
    base_retriever=vectordb.as_retriever(search_type="mmr")
)

In [ ]:
compressed_docs = compression_retriever.invoke(question)
pretty_print_docs(compressed_docs)


Document 1:

A manageable strategy


In [ ]:
from langchain.chains import RetrievalQA

In [ ]:
from langchain.prompts import PromptTemplate

# Build prompt
template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. Use three sentences maximum. Keep the answer as concise as possible. Always say "thanks for asking!" at the end of the answer. 
{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)


In [ ]:
qa_chain_mr = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    chain_type="refine"
)

In [ ]:
result = qa_chain_mr.invoke({"query": question})

In [ ]:
result["result"]

'In the context of web design and development, a manageable strategy involves breaking down tasks into smaller, more manageable chunks to adapt and evolve the design system over time. It is crucial to establish a clear governance plan to address any issues or challenges that may arise, such as when existing patterns do not work for specific applications. By having protocols in place to modify patterns, recommend alternative solutions, or create new patterns when needed, the design system can remain flexible and responsive to user needs. This approach ensures that the design system continues to be a helpful and valuable tool for designers and developers.'

In [ ]:
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

In [ ]:
from langchain.chains import ConversationalRetrievalChain
retriever=vectordb.as_retriever()
qa = ConversationalRetrievalChain.from_llm(
    llm,
    retriever=retriever,
    memory=memory
)

In [ ]:
result = qa.invoke({"question": question})

In [ ]:
result['answer']

'A manageable strategy, in the context of the text provided, refers to the approach organizations are taking to address the challenges posed by the multi-device universe in web design. It involves breaking down the responsibilities of ensuring web experiences work well across various devices into smaller, more manageable tasks. This strategy aims to help maintain sanity and efficiency in dealing with the complexities of designing for different devices.'

In [ ]:
question = "why A manageable strategy needed?"
result = qa.invoke({"question": question})

In [ ]:
result['answer']

'A manageable strategy is needed in addressing the challenges posed by the multi-device universe in web design because the web landscape is continuously evolving with new devices and technologies. Designing for a wide range of devices such as smartphones, tablets, phablets, netbooks, notebooks, desktops, TVs, game consoles, and more requires breaking down responsibilities into smaller, more manageable tasks. This approach helps maintain sanity and efficiency in creating web experiences that look and function beautifully across various devices and screen sizes. Additionally, organizations are realizing that continuously overhauling their websites every few years is not an optimal solution in this rapidly changing digital environment.'